# Analysis of bike traffic on Fremont Bridge in Seattle

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

### Data Preprocessing

In [29]:
bike_traffic_data = pd.read_csv('raw_bike_traffic_data.csv')
bike_traffic_data.index = pd.to_datetime(bike_traffic_data['Date'])
bike_traffic_data.drop(columns=['Date'], inplace=True)
print(bike_traffic_data.index.dtype)
print(bike_traffic_data.index.is_unique)
print(bike_traffic_data.index.has_duplicates)
print(bike_traffic_data.index.isna().sum())

bike_traffic_data.rename(columns={'Fremont Bridge Total': 'Total', 'Fremont Bridge East Sidewalk': 'East', 'Fremont Bridge West Sidewalk': 'West'}, inplace=True)

styled_bike_traffic_data = bike_traffic_data.style.format(precision=0).hide(bike_traffic_data.index[3:-3], axis=0)
styled_bike_traffic_data

/var/folders/pm/cppmzmm510j_2083hwms8rdh0000gn/T/ipykernel_20576/1226459397.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  bike_traffic_data.index = pd.to_datetime(bike_traffic_data['Date'])


datetime64[ns]
True
False
0


,Total,East,West
Date,,,
2012-10-03 00:00:00,13,4,9
2012-10-03 01:00:00,10,4,6
2012-10-03 02:00:00,2,1,1
2023-01-31 21:00:00,22,3,19
2023-01-31 22:00:00,18,7,11
2023-01-31 23:00:00,6,3,3
